In [1]:
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph
from openai import OpenAI
from langchain_community.chat_models import ChatOpenAI

# Définition de l'état
class CVGenerationState(TypedDict):
    profile: dict             # Données du profil
    job: dict                # Données sur le job
    current_step: str
    error: Optional[str]

# Exemple de structure pour les données
# profile = {
#     "head_raw": dict,           # Informations de l'en-tête
#     "skills_raw": dict,         # Compétences
#     "experiences": List[dict],  # Liste d'expériences
#     "education": List[dict],    # Liste de formations
#     "hobbies_raw": dict         # Loisirs
# }
# job = {
#     "job_posting": str,     # Fiche de poste
#     "job_summary": Optional[str] # Résumé de la fiche de poste
# }

# Structure pour une expérience dans profile["experiences"]
# experience = {
#     "titre_raw": str,
#     "titre_refined": str,
#     "dates_raw": str,
#     "dates_refined": str,
#     "entreprise_raw": str,
#     "entreprise_refined": str,
#     "lieu_raw": str,
#     "lieu_refined": str,
#     "description": str,         # Description brute
#     "sumup": str,              # Résumé
#     "bullets_points": List[str], # Points clés
#     "poid": float              # Poids en %
# }

# Structure pour une formation dans profile["education"]
# education = {
#     "titre_raw": str,
#     "titre_refined": str,
#     "dates_raw": str,
#     "dates_refined": str,
#     "etablissement_raw": str,
#     "etablissement_refined": str,
#     "lieu_raw": str,
#     "lieu_refined": str,
#     "description": str,         # Description brute
#     "description_refined": str, # Description raffinée
#     "sumup": str,              # Résumé
#     "poid": float              # Poids en %
# }


In [2]:
# Définition du modèle de langage
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# Test du modèle
test_response = llm.invoke("Dis bonjour en français")
print("Test de réponse du modèle:")
print(test_response.content)


/var/folders/kz/q2b2x49j16x2jfrqxg8_grdc0000gn/T/ipykernel_14210/2207125639.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Test de réponse du modèle:
Bonjour !


In [3]:
import json

# Chargement des données de test depuis le fichier JSON
with open('data_test.json', 'r', encoding='utf-8') as json_file:
    test_data = json.load(json_file)

# Chargement de la fiche de poste depuis le fichier texte
with open('fiche_poste.txt', 'r', encoding='utf-8') as text_file:
    job_posting = text_file.read()

# Initialisation de l'état avec les données de test et la fiche de poste
initial_state = {
    "profile": {
        "head_raw": test_data["cv_data"]["head"],
        "skills_raw": test_data["cv_data"]["skills"],
        "experiences": [
            {
                "titre_raw": exp["intitule"],
                "titre_refined": None,
                "dates_raw": exp["dates"],
                "dates_refined": None,
                "entreprise_raw": exp["etablissement"],
                "entreprise_refined": None,
                "lieu_raw": exp["lieu"],
                "lieu_refined": None,
                "description": exp["description"],
                "sumup": None,
                "bullets_points": [],
                "poid": None
            }
            for exp in test_data["cv_data"]["experiences"]["experiences"]
        ],
        "education": [
            {
                "titre_raw": edu["intitule"],
                "titre_refined": None,
                "dates_raw": edu["dates"],
                "dates_refined": None,
                "etablissement_raw": edu["etablissement"],
                "etablissement_refined": None,
                "lieu_raw": edu["lieu"],
                "lieu_refined": None,
                "description_raw": edu["description"],
                "description_refined": None,
                "sumup": None,
                "poid": None
            }
            for edu in test_data["cv_data"]["education"]["educations"]
        ],
        "hobbies_raw": test_data["cv_data"]["hobbies"]
    },
    "job": {
        "job_posting": job_posting,
        "job_summary": None
    }
}

In [4]:
# Nodes (fonctions de transformation)
def summarize_job(state: CVGenerationState) -> CVGenerationState:
    response = llm.invoke(
        "Faites un résumé de cette fiche de poste en 100 mots maximum, en incluant les informations principales pour rédiger un CV.\n\n" + 
        state["job"]["job_posting"]
    )
    state["job"]["job_summary"] = response.content.strip()
    return state


In [5]:
initial_state = summarize_job(initial_state)

# on affiche le résumé de la fiche de poste
print(initial_state["job"]["job_summary"])


**Résumé de la fiche de poste - Machine Learning Engineer**

Poste : Machine Learning Engineer en CDI, Paris (hybride). Expérience requise : 3+ ans. Salaire : 55k - 75k €. Startup innovante en IA, spécialisée dans le Machine Learning et Deep Learning. Missions : concevoir et déployer des modèles ML, optimiser pipelines de données, développer des API, et collaborer avec des équipes pluridisciplinaires. Compétences requises : maîtrise de Python (TensorFlow, PyTorch), SQL, AWS, GCP, MLOps (MLflow, Kubeflow), et développement (FastAPI, Flask). Diplôme en Informatique ou équivalent souhaité. Environnement dynamique et projets innovants. Postuler à recrutement@startupAI.com.


In [6]:
def summarize_profile(state: CVGenerationState) -> CVGenerationState:
    # Résumer chaque expérience
    for experience in state["profile"]["experiences"]:
        response = llm.invoke(
            "Faites un résumé de cette expérience en 50 mots maximum.\n\n" + 
            experience["description"]
        )
        experience["sumup"] = response.content.strip()
    
    # Résumer chaque formation
    for education in state["profile"]["education"]:
        response = llm.invoke(
            "Faites un résumé de cette formation en 50 mots maximum.\n\n" + 
            education["description_raw"]
        )
        education["sumup"] = response.content.strip()
    
    return state

#tester la fonction sur initial_state
initial_state = summarize_profile(initial_state)

#afficher les résumés
for experience in initial_state["profile"]["experiences"]:
    print("Résumé de l'expérience :", experience.get("sumup", "Pas de résumé disponible."))


Résumé de l'expérience : Lors de ma mission chez Renault, j'ai modernisé les équipes de data engineering, piloté la transition vers dbt sur Cloud Run, et introduit des pratiques DevOps. J'ai également développé des outils pour faciliter l'accès aux données et créé des modèles de Machine Learning pour estimer les valeurs résiduelles des véhicules.
Résumé de l'expérience : En tant que co-fondateur et CTO de Kadi, j'ai dirigé le développement d'une application mobile innovante pour l'analyse automatique de tickets de caisse, intégrant IA et cloud. J'ai supervisé l'architecture technique, le déploiement sur les stores, et optimisé des pipelines de données, garantissant une expérience utilisateur fluide.
Résumé de l'expérience : Dans une entreprise familiale de prothèses médicales, j'ai modernisé les processus internes par la digitalisation des bases de données, l'optimisation de la fabrication via un algorithme génétique, et l'automatisation des tests. Ces projets ont amélioré l'efficacité

In [7]:
#afficher les résumés des éducations
for education in initial_state["profile"]["education"]:
    print("Résumé de l'éducation :", education.get("sumup", "Pas de résumé disponible."))


Résumé de l'éducation : À l’École des Ponts ParisTech, j'ai acquis une formation scientifique et technique de haut niveau, incluant un tronc commun en mathématiques, physique, informatique et sciences de l’ingénieur, suivi d'une spécialisation en supply chain et machine learning durant les deux dernières années.
Résumé de l'éducation : Cette année de spécialisation en ingénierie mathématique a permis d'approfondir les mathématiques appliquées, la finance computationnelle, le machine learning et la gestion de la chaîne d'approvisionnement, tout en apprenant l'italien. Les cours principaux incluent la finance computationnelle et le machine learning, avec un accent sur les méthodes quantitatives et les algorithmes.
Résumé de l'éducation : J'ai suivi une formation académique rigoureuse en classes préparatoires pendant deux ans, axée sur la préparation aux concours des grandes écoles d'ingénieurs, développant ainsi mes compétences en sciences et en mathématiques. Cette expérience m'a permis

In [8]:
def select_experiences(state: CVGenerationState) -> CVGenerationState:
    # Préparer le contexte pour l'IA
    job_context = state["job"]["job_summary"]
    experiences_context = "\n".join([
        f"- {exp['titre_raw']} à {exp['lieu_raw']} ({exp['dates_raw']}) chez {exp['entreprise_raw']}: {exp['sumup']}" 
        for exp in state["profile"]["experiences"]
    ])
    
    # Premier LLM pour sélectionner et pondérer les expériences
    prompt = f"""
    En tant qu'expert RH, analysez les expériences du candidat par rapport au poste.
    Sélectionnez les expériences les plus pertinentes et attribuez-leur un pourcentage d'importance (la somme doit faire 100%).
    Répondez sous forme de texte simple en listant les expériences retenues avec leur pourcentage.
    
    Poste à pourvoir:
    {job_context}
    
    Expériences du candidat:
    {experiences_context}
    """
    
    selection_response = llm.invoke(prompt)
    selection_text = selection_response.content
    
    # Pour chaque expérience, demander au second LLM d'extraire le poids
    for exp in state["profile"]["experiences"]:
        exp_prompt = f"""
        Voici la sélection d'expériences pertinentes avec leurs pourcentages:
        {selection_text}
        
        Pour l'expérience suivante, retournez uniquement le pourcentage attribué (juste le nombre), ou "null" si elle n'est pas mentionnée:
        {exp['titre_raw']} chez {exp['entreprise_raw']}
        """
        
        weight_response = llm.invoke(exp_prompt)
        try:
            weight = float(weight_response.content.strip())
            exp["poid"] = weight
        except ValueError:
            exp["poid"] = None
    
    return state

# Tester la fonction
initial_state = select_experiences(initial_state)

# Afficher les expériences sélectionnées
print("\nExpériences sélectionnées et leur poids:")
for exp in initial_state["profile"]["experiences"]:
    weight = exp.get("poid")
    if weight is not None:
        print(f"- {exp['titre_raw']} chez {exp['entreprise_raw']}: {weight}%")
    else:
        print(f"- {exp['titre_raw']} chez {exp['entreprise_raw']}: Non retenue")



Expériences sélectionnées et leur poids:
- Senior Data Scientist chez EY: 40.0%
- Co-fondateur et CTO chez Kadi: 30.0%
- Ingénieur en Transformation Digitale chez Blispac: 15.0%
- Consultant en Stratégie et Supply Chain chez Diagma: Non retenue
- Analyste de Marché chez Total: 10.0%
- Coordinateur de Projets chez L’Œuvre d’Orient: Non retenue
- Consultant junior chez Cdiscount: 5.0%


In [9]:
def select_education(state):
    """
    Sélectionne et pondère les formations pertinentes pour le poste.
    """
    # Récupérer le contexte
    job_context = state["job"]["job_summary"]
    education_context = "\n".join([f"{edu['titre_raw']} - {edu['etablissement_raw']} ({edu['dates_raw']}) {edu['lieu_raw']} {edu['sumup']}" 
                                 for edu in state["profile"]["education"]])
    
    # Créer le prompt
    prompt = f"""
    En tant qu'expert RH, analysez les formations du candidat et sélectionnez celles qui sont les plus pertinentes 
    pour le poste à pourvoir. Attribuez un pourcentage à chaque formation selon sa pertinence.

    Formations du candidat:
    {education_context}

    Poste à pourvoir:
    {job_context}
    """
    
    selection_response = llm.invoke(prompt)
    selection_text = selection_response.content
    
    # Pour chaque formation, demander au LLM d'extraire le poids
    for edu in state["profile"]["education"]:
        edu_prompt = f"""
        Voici la sélection de formations pertinentes avec leurs pourcentages:
        {selection_text}
        
        Pour la formation suivante, retournez uniquement le pourcentage attribué (juste le nombre), ou "null" si elle n'est pas mentionnée:
        {edu['titre_raw']} à {edu['etablissement_raw']}
        """
        
        weight_response = llm.invoke(edu_prompt)
        try:
            weight = float(weight_response.content.strip())
            edu["poid"] = weight
        except ValueError:
            edu["poid"] = None
    
    return state

# Tester la fonction
initial_state = select_education(initial_state)

# Afficher les formations sélectionnées
print("\nFormations sélectionnées et leur poids:")
for edu in initial_state["profile"]["education"]:
    weight = edu.get("poid")
    if weight is not None:
        print(f"- {edu['titre_raw']} à {edu['etablissement_raw']}: {weight}%")
    else:
        print(f"- {edu['titre_raw']} à {edu['etablissement_raw']}: Non retenue")



Formations sélectionnées et leur poids:
- Diplôme d’ingénieur à École des Ponts ParisTech: 40.0%
- Master 2 en Mathematical Engineering à Politecnico di Milano: 45.0%
- Classe préparatoire scientifique à Collège Stanislas Paris: 15.0%


In [10]:
def adjust_weights(state, threshold=10):
    """
    Ajuste les poids des expériences et formations :
    - Met à 0 les poids inférieurs ou égaux au seuil 
    - Rééquilibre les poids restants pour garder une somme de 100%
    """
    # Ajuster les poids des expériences
    if "profile" in state and "experiences" in state["profile"]:
        # Mettre 0 pour les poids None ou non numériques
        for exp in state["profile"]["experiences"]:
            try:
                if exp.get("poid") is None or not isinstance(exp.get("poid"), (int, float)):
                    exp["poid"] = 0
            except:
                exp["poid"] = 0

        exp_weights = [exp["poid"] for exp in state["profile"]["experiences"]]
        total_weight = sum(w for w in exp_weights if w > threshold)
        
        if total_weight > 0:
            for exp in state["profile"]["experiences"]:
                if exp["poid"] <= threshold:
                    exp["poid"] = 0
                else:
                    exp["poid"] = round((exp["poid"] / total_weight) * 100)

    # Ajuster les poids des formations
    if "profile" in state and "education" in state["profile"]:
        # Mettre 0 pour les poids None ou non numériques
        for edu in state["profile"]["education"]:
            try:
                if edu.get("poid") is None or not isinstance(edu.get("poid"), (int, float)):
                    edu["poid"] = 0
            except:
                edu["poid"] = 0

        edu_weights = [edu["poid"] for edu in state["profile"]["education"]]
        total_weight = sum(w for w in edu_weights if w > threshold)
        
        if total_weight > 0:
            for edu in state["profile"]["education"]:
                if edu["poid"] <= threshold:
                    edu["poid"] = 0
                else:
                    edu["poid"] = round((edu["poid"] / total_weight) * 100)
    
    return state

# Tester la fonction
initial_state = adjust_weights(initial_state)


In [11]:
def generate_experience_bullets(state):
    """
    Génère des bullet points pour chaque expérience en fonction de leur poids
    """
    if "profile" not in state or "experiences" not in state["profile"]:
        return state
    
    for exp in state["profile"]["experiences"]:
        # Vérifier que poid existe et n'est pas None avant de l'utiliser
        if not exp.get("poid"):
            continue
            
        # Calculer le nombre de bullet points en fonction du poids
        nb_bullets = max(2, min(6, round(exp["poid"] / 20)))
        
        prompt = f"""
        En tant qu'expert RH, générez {nb_bullets} bullet points percutants qui mettent en valeur cette expérience 
        professionnelle par rapport au poste visé.
        
        Chaque bullet point doit:
        - Commencer par un verbe d'action fort
        - Être concis et impactant
        - Mettre en avant les réalisations concrètes
        - Être adapté au poste visé
        
        Répondez au format JSON suivant:
        {{
            "bullet_points": [
                "Premier bullet point",
                "Deuxième bullet point",
                etc.
            ]
        }}
        
        Poste visé (résumé):
        {state["job"]["job_summary"]}
        
        Expérience à décrire:
        {exp["description"]}
        """
        
        response = llm.invoke(input=prompt, response_format={"type": "json_object"})
        bullets = json.loads(response.content)
        
        # Ajouter les bullet points à l'expérience
        exp["bullets_points"] = bullets["bullet_points"]
    
    return state

# Tester la fonction
initial_state = generate_experience_bullets(initial_state)

# Afficher les bullet points générés
print("\nBullet points générés pour chaque expérience:")
for exp in initial_state["profile"]["experiences"]:
    if exp.get("poid", 0) > 0:
        print(f"\n{exp['titre_refined']} chez {exp['entreprise_refined']}:")
        for bullet in exp["bullets_points"]:
            print(f"• {bullet}")



Bullet points générés pour chaque expérience:

None chez None:
• Optimisé les pipelines de données en concevant et déployant une solution dbt sur Cloud Run, améliorant la qualité et la rapidité des processus de transformation des données, approuvée par la direction technique.
• Dirigé une équipe de 5 data engineers pour moderniser les pratiques de data engineering, intégrant des méthodologies agiles et des outils modernes, ce qui a permis une adoption généralisée de dbt comme standard au sein de Renault.

None chez None:
• Conçu et déployé une application mobile innovante sur GCP, intégrant des modèles de Machine Learning pour l'analyse automatique de tickets de caisse, validée par des utilisateurs sur les principales plateformes mobiles.
• Optimisé des pipelines de données évolutifs en intégrant des pratiques MLOps, garantissant la scalabilité et la haute disponibilité des services, tout en améliorant la performance des modèles de détection d'objets en production.

None chez None:
• 

In [12]:
def generate_education_description(state):
    """
    Génère une description raffinée pour chaque formation en fonction de leur poids
    """
    if "profile" not in state or "education" not in state["profile"]:
        return state
    
    for edu in state["profile"]["education"]:
        if edu.get("poid", 0) == 0:
            continue
            
        # Adapter la longueur de la description en fonction du poids
        nb_mots = max(15, min(50, round(edu["poid"] / 10)))
        
        prompt = f"""
        En tant qu'expert RH, générez une description percutante de cette formation
        qui met en valeur sa pertinence par rapport au poste visé.
        
        La description doit:
        - Être concise (environ {nb_mots} mots)
        - Mettre en avant les compétences acquises
        - Souligner les réalisations académiques importantes
        - Être adaptée au poste visé
        
        Poste visé (résumé):
        {state["job"]["job_summary"]}
        
        Formation à décrire:
        {edu["description_raw"]}
        """
        
        response = llm.invoke(input=prompt)
        
        # Ajouter la description raffinée à la formation
        edu["description_refined"] = response.content
    
    return state

# Tester la fonction
initial_state = generate_education_description(initial_state)

# Afficher les descriptions générées
print("\nDescriptions raffinées pour chaque formation:")
for edu in initial_state["profile"]["education"]:
    if edu.get("poid", 0) > 0:
        print(f"\n{edu['titre_raw']} - {edu['etablissement_raw']}:")
        print(edu["description_refined"])



Descriptions raffinées pour chaque formation:

Diplôme d’ingénieur - École des Ponts ParisTech:
Formation à l’École des Ponts ParisTech : expertise en machine learning, optimisation de données et modélisation avancée.

Master 2 en Mathematical Engineering - Politecnico di Milano:
Formation en ingénierie mathématique : compétences avancées en machine learning et finance computationnelle, essentielles pour le poste.

Classe préparatoire scientifique - Collège Stanislas Paris:
Formation rigoureuse en classes préparatoires, développant compétences analytiques et techniques essentielles pour le Machine Learning.


In [13]:
print("\nÉtat actuel:")
print(json.dumps(initial_state, indent=2, ensure_ascii=False))



État actuel:
{
  "profile": {
    "head_raw": {
      "general_title": "Data Engineer / Data Scientist | Expert Cloud et Data. Alexis de Monts est un ingénieur des Ponts et Chaussées, spécialisé dans la transformation numérique des entreprises grâce à ses compétences en Data Science. Avec une formation scientifique généraliste et une spécialisation en Data Science au Politecnico di Milano, il se concentre sur l'intégration de solutions Cloud et d'algorithmes de Machine Learning dans des systèmes informatiques complexes. Son expérience inclut la création d'un Data Hub pour Renault et le développement d'applications mobiles innovantes.",
      "email": "alexis.demonts.s@gmail.com",
      "phone": "07 81 37 86 80",
      "name": "Alexis de Monts"
    },
    "skills_raw": {
      "description": "Alexis possède une expertise approfondie dans plusieurs domaines clés. En tant que Data Engineer et Data Scientist, il maîtrise les technologies Cloud, notamment Google Cloud Platform (GCP) et Ama